# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Business Problem
Computing Vision (a made-up company for the purposes of this project) sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t have much background in creating movies. You are charged with exploring what types of films are currently doing the best at the box office using different samples of available data. You then will translate those findings into actionable insights that the head of Computing Vision's new movie studio can use to help decide what type of films to create.

# EDA (Each data set)

### 1. Box Office Mojo

In [2]:
box = pd.read_csv('bom.movie_gross.csv')
box.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [3]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### 2.  The Movie Database

In [4]:
tmdb = pd.read_csv('tmdb.movies.csv')
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [5]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


# 3. The Numbers *

In [4]:
tn = pd.read_csv('tn.movie_budgets.csv')
tn.head(10)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
5,6,"Dec 18, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,053,311,220"
6,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200"
7,8,"May 24, 2007",Pirates of the Caribbean: At Worldâs End,"$300,000,000","$309,420,425","$963,420,425"
8,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209"
9,10,"Nov 6, 2015",Spectre,"$300,000,000","$200,074,175","$879,620,923"


In [5]:
#tn['domestic_gross'] = pd.to_numeric(tn['domestic_gross'])
#tn['domestic_gross'] = tn['domestic_gross'].astype(str).astype(int)
tn[tn.columns[3:]] = (tn[tn.columns[3:]].replace('[\$,]', '', regex=True).astype(float))/1000000
tn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


In [6]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
q = """SELECT movie, release_date, worldwide_gross
     FROM tn
     ORDER BY worldwide_gross DESC
     LIMIT 10;"""

movies = pysqldf(q)
movies

,movie,release_date,worldwide_gross
0,Avatar,"Dec 18, 2009",2776.345279
1,Titanic,"Dec 19, 1997",2208.208395
2,Star Wars Ep. VII: The Force Awakens,"Dec 18, 2015",2053.311220
3,Avengers: Infinity War,"Apr 27, 2018",2048.134200
4,Jurassic World,"Jun 12, 2015",1648.854864
5,Furious 7,"Apr 3, 2015",1518.722794
6,The Avengers,"May 4, 2012",1517.935897
7,Avengers: Age of Ultron,"May 1, 2015",1403.013963
8,Black Panther,"Feb 16, 2018",1348.258224
9,Harry Potter and the Deathly Hallows: Part II,"Jul 15, 2011",1341.693157


### 4. Rotten Tomatoes

In [9]:
rt = pd.read_csv("rt.movie_info.tsv", sep='\t')
rt.head(3)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN


# 5. IMBD (SQL) *

In [7]:
import sqlite3 
conn = sqlite3.connect('im.db')

In [8]:
q= """
SELECT original_title AS movie, start_year, genres
FROM movie_basics
"""
im = pd.read_sql(q, conn)
im.head()

,movie,start_year,genres
0,Sunghursh,2013,"Action,Crime,Drama"
1,Ashad Ka Ek Din,2019,"Biography,Drama"
2,The Other Side of the Wind,2018,Drama
3,Sabse Bada Sukh,2018,"Comedy,Drama"
4,La Telenovela Errante,2017,"Comedy,Drama,Fantasy"


In [10]:
#im[im['genres']] = (im['genres']).astype(str)

In [11]:
im.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   movie       146123 non-null  object
 1   start_year  146144 non-null  int64 
 2   genres      140736 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.3+ MB


In [79]:
#pd.read_sql("""
#SELECT *
#FROM movie_basics
#WHERE original_title LIKE '%Avatar%'
#""", conn)

# Merging IMBD with The Numbers

In [33]:
df_combine_genres = pd.merge(tn, im, how = 'inner')
#df_combine_genres.head()
#df_combine_genres.info()

In [34]:
df_combine_genres.sort_values(['worldwide_gross'], ascending=[False], inplace= True)
#df_combine_genres.set_index('genres',  inplace= True)
df_combine_genres.head(15)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,start_year,genres
41,43,"Dec 19, 1997",Titanic,200.0,659.363944,2208.208395,2012,Adventure
42,43,"Dec 19, 1997",Titanic,200.0,659.363944,2208.208395,2018,Family
3,7,"Apr 27, 2018",Avengers: Infinity War,300.0,678.815482,2048.134200,2018,"Action,Adventure,Sci-Fi"
24,34,"Jun 12, 2015",Jurassic World,215.0,652.270625,1648.854864,2015,"Action,Adventure,Sci-Fi"
18,27,"May 4, 2012",The Avengers,225.0,623.279547,1517.935897,2012,"Action,Adventure,Sci-Fi"
2,4,"May 1, 2015",Avengers: Age of Ultron,330.6,459.005868,1403.013963,2015,"Action,Adventure,Sci-Fi"
40,42,"Feb 16, 2018",Black Panther,200.0,700.059566,1348.258224,2018,"Action,Adventure,Sci-Fi"
110,13,"Jun 22, 2018",Jurassic World: Fallen Kingdom,170.0,417.719760,1305.772799,2018,"Action,Adventure,Sci-Fi"
147,56,"Nov 22, 2013",Frozen,150.0,400.738009,1272.469910,2010,"Adventure,Drama,Sport"
148,56,"Nov 22, 2013",Frozen,150.0,400.738009,1272.469910,2013,"Adventure,Animation,Comedy"


In [36]:
#df_combine_genres.str.contains('Sci-Fi', case=True)
#'Sci-Fi' in df_combine_genres.index

sfi = df_combine_genres['genres'].astype(str).str.contains('Sci-Fi', regex = True, na = False)
df_combine_genres[sfi].sum()

id                                                               11432
release_date         Apr 27, 2018Jun 12, 2015May 4, 2012May 1, 2015...
movie                Avengers: Infinity WarJurassic WorldThe Avenge...
production_budget                                                13893
domestic_gross                                                 17752.9
worldwide_gross                                                47951.9
start_year                                                      435159
genres               Action,Adventure,Sci-FiAction,Adventure,Sci-Fi...
dtype: object

In [16]:
#df = df.set_index([pd.Index(['A', 'B', 'C', 'D', 'E', 'F', 'G'])])
#sg=[]

#sg=df_combine_genres['worldwide_gross'].loc[['Sci-Fi']].sum() #sum of every movie that is at least Sci-fi
#sg

In [38]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

q = """SELECT movie, worldwide_gross, genres
     FROM df_combine_genres
     WHERE genres LIKE '%Sci-Fi%'
     ;"""
gen = pysqldf(q)
gen

,movie,worldwide_gross,genres
0,Avengers: Infinity War,2048.134200,"Action,Adventure,Sci-Fi"
1,Jurassic World,1648.854864,"Action,Adventure,Sci-Fi"
2,The Avengers,1517.935897,"Action,Adventure,Sci-Fi"
3,Avengers: Age of Ultron,1403.013963,"Action,Adventure,Sci-Fi"
4,Black Panther,1348.258224,"Action,Adventure,Sci-Fi"
...,...,...,...
211,The Dead Undead,0.000000,"Action,Horror,Sci-Fi"
212,Vessel,0.000000,Sci-Fi
213,Infected,0.000000,"Action,Horror,Sci-Fi"
214,Mutant World,0.000000,Sci-Fi


In [40]:
from pandasql import sqldf
pysqldf = lambda f: sqldf(f, globals())
f = """
     SELECT SUM(worldwide_gross) SciFi
     FROM df_combine_genres
     WHERE genres LIKE 'Sci-Fi'
     ;"""

Ugen = pysqldf(f)
Ugen

,SciFi
0,1911.578804


In [20]:
df_combine_genres['genres']
genre_list = df_combine_genres['genres'].tolist()
genre_list

['Adventure',
 'Family',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Adventure,Drama,Sport',
 'Adventure,Animation,Comedy',
 'Family,Fantasy,Musical',
 'Family,Fantasy,Musical',
 'Action,Adventure,Animation',
 'Action,Crime,Thriller',
 'Adventure,Animation,Comedy',
 'Action,Adventure,Fantasy',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Sci-Fi',
 'Action,Adventure,Thriller',
 'Action,Adventure,Sci-Fi',
 'Action,Thriller',
 'Adventure,Animation,Comedy',
 'Action,Adventure,Fantasy',
 'Adventure,Animation,Comedy',
 'Documentary',
 'Fantasy,Musical',
 'Adventure,Family,Fantasy',
 'Adventure,Animation,Comedy',
 'Adventure,Animation,Comedy',
 'Adventure,Family,Fantasy',
 'Adventure,Animation,Drama',
 'Adventure,Animation,Comedy',
 'Action,Adventure,Comedy',
 'Animation',
 'Adventure,Drama,Family',
 'Adventure,Fantasy',
 'Adventure,Fantas

In [182]:
#genres=['Action', 'Adventure', 'Fantasy', 'Drama', 'Sci-Fi', 'Comedy']

In [21]:
df_genres = df_combine_genres[df_combine_genres['genres'].isin(['Action', 'Sci-Fi'])]
df_genres

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,start_year,genres
145,55,"Jun 2, 2017",Wonder Woman,150.0,412.563408,821.133378,2014,Sci-Fi
191,36,"Jun 29, 2005",War of the Worlds,132.0,234.280354,606.836535,2013,Sci-Fi
490,37,"Jun 27, 2008",Wanted,75.0,134.508551,342.416460,2010,Action
2325,56,"Dec 21, 2016",Dangal,9.5,12.391761,294.654618,2013,Action
283,80,"Jun 1, 1990",Total Recall,65.0,119.394839,261.400000,2010,Action
...,...,...,...,...,...,...,...,...
2101,86,"Jun 23, 2015",Crossroads,0.5,0.000000,0.000000,2014,Action
2541,91,"Feb 9, 2010",Icarus,6.0,0.000000,0.000000,2017,Sci-Fi
3276,71,"Jan 9, 2015",Vessel,0.8,0.000000,0.000000,2013,Sci-Fi
3133,17,"Sep 8, 2015",Checkmate,1.5,0.000000,0.000000,2016,Action


In [22]:
df_genres = df_combine_genres[df_combine_genres['genres'].df_combine_genres.str.contains('Sci-Fi')]
df_genres

AttributeError: 'Series' object has no attribute 'df_combine_genres'